In [70]:
import pandas as pd
import csv

from sqlalchemy import create_engine
from sqlalchemy import delete

In [71]:
from datetime import datetime

In [72]:
import sqlite3
import os
import glob

## Getting data from CSVs to DB

In [ ]:
sample2020 = pd.read_csv('./VAERS-data/2020VAERSDATA/2020VAERSDATA.csv', encoding_errors = 'ignore', low_memory=False)

In [ ]:
sample2020.head()

https://github.com/kevinchiv/Predicting-Kickstarter-Success/blob/master/00%20-%20PostgreSQL%20Through%20Jupyter.ipynb
https://www.w3schools.com/sql/sql_datatypes.asp
https://datatofish.com/pandas-dataframe-to-sql/
https://datatofish.com/create-database-python-using-sqlite3/
https://dev.mysql.com/doc/connector-python/en/connector-python-api-mysqlcursor-execute.html
https://chartio.com/resources/tutorials/how-to-insert-if-row-does-not-exist-upsert-in-mysql/

In [73]:
def new_table_statement(df, table_name):
    
    column_names = df.columns.str.strip()
    datatypes = df.dtypes.values
    
    statement = 'CREATE TABLE IF NOT EXISTS ' + table_name + ' (\n'
    
    for idx, column_name in enumerate(column_names):
        dtype = datatypes[idx]
        statement += column_name.lower()
        if dtype == 'float64':
            statement += ' DECIMAL'
        elif dtype =='int64':
            statement += ' INTEGER'
        elif dtype == 'O':
            statement += ' VARCHAR'
        #denote primary key
        #if column_name == 'VAERS_ID':
            #statement += ' PRIMARY KEY'
        if idx != len(column_names) - 1:
            statement += ", \n"
    statement += ');'
    print(statement)
    return statement

In [74]:
new_table_statement(sample2020, 'test')

CREATE TABLE IF NOT EXISTS test (
vaers_id INTEGER, 
recvdate VARCHAR, 
state VARCHAR, 
age_yrs DECIMAL, 
cage_yr DECIMAL, 
cage_mo DECIMAL, 
sex VARCHAR, 
rpt_date VARCHAR, 
symptom_text VARCHAR, 
died VARCHAR, 
datedied VARCHAR, 
l_threat VARCHAR, 
er_visit VARCHAR, 
hospital VARCHAR, 
hospdays DECIMAL, 
x_stay VARCHAR, 
disable VARCHAR, 
recovd VARCHAR, 
vax_date VARCHAR, 
onset_date VARCHAR, 
numdays DECIMAL, 
lab_data VARCHAR, 
v_adminby VARCHAR, 
v_fundby VARCHAR, 
other_meds VARCHAR, 
cur_ill VARCHAR, 
history VARCHAR, 
prior_vax VARCHAR, 
splttype VARCHAR, 
form_vers INTEGER, 
todays_date VARCHAR, 
birth_defect VARCHAR, 
ofc_visit VARCHAR, 
er_ed_visit VARCHAR, 
allergies VARCHAR);


'CREATE TABLE IF NOT EXISTS test (\nvaers_id INTEGER, \nrecvdate VARCHAR, \nstate VARCHAR, \nage_yrs DECIMAL, \ncage_yr DECIMAL, \ncage_mo DECIMAL, \nsex VARCHAR, \nrpt_date VARCHAR, \nsymptom_text VARCHAR, \ndied VARCHAR, \ndatedied VARCHAR, \nl_threat VARCHAR, \ner_visit VARCHAR, \nhospital VARCHAR, \nhospdays DECIMAL, \nx_stay VARCHAR, \ndisable VARCHAR, \nrecovd VARCHAR, \nvax_date VARCHAR, \nonset_date VARCHAR, \nnumdays DECIMAL, \nlab_data VARCHAR, \nv_adminby VARCHAR, \nv_fundby VARCHAR, \nother_meds VARCHAR, \ncur_ill VARCHAR, \nhistory VARCHAR, \nprior_vax VARCHAR, \nsplttype VARCHAR, \nform_vers INTEGER, \ntodays_date VARCHAR, \nbirth_defect VARCHAR, \nofc_visit VARCHAR, \ner_ed_visit VARCHAR, \nallergies VARCHAR);'

In [75]:
# https://stackoverflow.com/questions/9234560/find-all-csv-files-in-a-directory-using-python

def find_csvs(year):
    file_dir = os.path.abspath('.')
    data_folder = 'VAERS-data'
    data_year_folder = str(year) + 'VAERSData' #usual format
    path = os.path.join(file_dir, data_folder, data_year_folder)
    print(path)
    csv_list = list(filter(lambda x: '.csv' in x, os.listdir(path)))
    return csv_list, path

In [76]:
find_csvs(2020)

/Users/zliao/Desktop/Data Sci/Metis/Module 5 - Data Engineering/Project/VAERS-data/2020VAERSData


(['2020VAERSVAX.csv', '2020VAERSSYMPTOMS.csv', '2020VAERSDATA.csv'],
 '/Users/zliao/Desktop/Data Sci/Metis/Module 5 - Data Engineering/Project/VAERS-data/2020VAERSData')

In [77]:
def make_delete_statement(table_name, year):
    statement = 'DELETE FROM ' + table_name + ' WHERE year = ' + str(year) + ';'
    return statement

In [78]:
make_delete_statement('test', 2021)

'DELETE FROM test WHERE year = 2021;'

In [79]:
string = '2020VAERSVAX.csv'

In [80]:
string = string.split('/')[-1].split('.')[0]
table_name = string[9:].lower() 
print(table_name)

vax


In [89]:
#only current year gets updated 

def make_tables(connection, year):
    this_year = datetime.now().year 
    csv_list, path = find_csvs(year)
    c = conn.cursor()
    for csv_file in csv_list:
        csv_path = os.path.join(path, csv_file)
        df = pd.read_csv(csv_path, encoding_errors = 'ignore', low_memory=False) 
        #making table name based on csv name 
        string = csv_file.split('/')[-1].split('.')[0]
        table_name = string[9:].lower() 
        year = int(string[:4])
        df['YEAR'] = year
        
        c.execute(new_table_statement(df, table_name))
        conn.commit()
        
        if this_year == year:
            #remove the rows with that year and replace with this updated csv data
            delete_statement = make_delete_statement(table_name, year)
            c.execute(delete_statement)
            conn.commit()
            print('Deleted any current data from given year!')
            #then replace
            df.to_sql(table_name, conn, if_exists = 'append', index=False)
        else:
            df.to_sql(table_name, conn, if_exists = 'append', index=False)
        conn.commit()

        print('Done! Inserted ' + table_name)
    

### Adding tables

In [95]:
conn = sqlite3.connect('vaers.db')

In [83]:
make_tables(conn, 2020)

/Users/zliao/Desktop/Data Sci/Metis/Module 5 - Data Engineering/Project/VAERS-data/2020VAERSData
CREATE TABLE IF NOT EXISTS vax (
vaers_id INTEGER, 
vax_type VARCHAR, 
vax_manu VARCHAR, 
vax_lot VARCHAR, 
vax_dose_series VARCHAR, 
vax_route VARCHAR, 
vax_site VARCHAR, 
vax_name VARCHAR, 
year INTEGER);
Done! Inserted vax
CREATE TABLE IF NOT EXISTS symptoms (
vaers_id INTEGER, 
symptom1 VARCHAR, 
symptomversion1 DECIMAL, 
symptom2 VARCHAR, 
symptomversion2 DECIMAL, 
symptom3 VARCHAR, 
symptomversion3 DECIMAL, 
symptom4 VARCHAR, 
symptomversion4 DECIMAL, 
symptom5 VARCHAR, 
symptomversion5 DECIMAL, 
year INTEGER);
Done! Inserted symptoms
CREATE TABLE IF NOT EXISTS data (
vaers_id INTEGER, 
recvdate VARCHAR, 
state VARCHAR, 
age_yrs DECIMAL, 
cage_yr DECIMAL, 
cage_mo DECIMAL, 
sex VARCHAR, 
rpt_date VARCHAR, 
symptom_text VARCHAR, 
died VARCHAR, 
datedied VARCHAR, 
l_threat VARCHAR, 
er_visit VARCHAR, 
hospital VARCHAR, 
hospdays DECIMAL, 
x_stay VARCHAR, 
disable VARCHAR, 
recovd VARCHA

In [87]:
make_tables(conn, 2021)

/Users/zliao/Desktop/Data Sci/Metis/Module 5 - Data Engineering/Project/VAERS-data/2021VAERSData
CREATE TABLE IF NOT EXISTS vax (
vaers_id INTEGER, 
vax_type VARCHAR, 
vax_manu VARCHAR, 
vax_lot VARCHAR, 
vax_dose_series VARCHAR, 
vax_route VARCHAR, 
vax_site VARCHAR, 
vax_name VARCHAR, 
year INTEGER);
Done! Inserted vax
CREATE TABLE IF NOT EXISTS data (
vaers_id INTEGER, 
recvdate VARCHAR, 
state VARCHAR, 
age_yrs DECIMAL, 
cage_yr DECIMAL, 
cage_mo DECIMAL, 
sex VARCHAR, 
rpt_date VARCHAR, 
symptom_text VARCHAR, 
died VARCHAR, 
datedied VARCHAR, 
l_threat VARCHAR, 
er_visit VARCHAR, 
hospital VARCHAR, 
hospdays DECIMAL, 
x_stay VARCHAR, 
disable VARCHAR, 
recovd VARCHAR, 
vax_date VARCHAR, 
onset_date VARCHAR, 
numdays DECIMAL, 
lab_data VARCHAR, 
v_adminby VARCHAR, 
v_fundby VARCHAR, 
other_meds VARCHAR, 
cur_ill VARCHAR, 
history VARCHAR, 
prior_vax VARCHAR, 
splttype VARCHAR, 
form_vers INTEGER, 
todays_date VARCHAR, 
birth_defect VARCHAR, 
ofc_visit VARCHAR, 
er_ed_visit VARCHAR,

In [96]:
make_tables(conn, 2022)

/Users/zliao/Desktop/Data Sci/Metis/Module 5 - Data Engineering/Project/VAERS-data/2022VAERSData
CREATE TABLE IF NOT EXISTS data (
vaers_id INTEGER, 
recvdate VARCHAR, 
state VARCHAR, 
age_yrs DECIMAL, 
cage_yr DECIMAL, 
cage_mo DECIMAL, 
sex VARCHAR, 
rpt_date VARCHAR, 
symptom_text VARCHAR, 
died VARCHAR, 
datedied VARCHAR, 
l_threat VARCHAR, 
er_visit VARCHAR, 
hospital VARCHAR, 
hospdays DECIMAL, 
x_stay VARCHAR, 
disable VARCHAR, 
recovd VARCHAR, 
vax_date VARCHAR, 
onset_date VARCHAR, 
numdays DECIMAL, 
lab_data VARCHAR, 
v_adminby VARCHAR, 
v_fundby VARCHAR, 
other_meds VARCHAR, 
cur_ill VARCHAR, 
history VARCHAR, 
prior_vax VARCHAR, 
splttype VARCHAR, 
form_vers INTEGER, 
todays_date VARCHAR, 
birth_defect VARCHAR, 
ofc_visit VARCHAR, 
er_ed_visit VARCHAR, 
allergies VARCHAR, 
year INTEGER);
Done! Inserted data
CREATE TABLE IF NOT EXISTS symptoms (
vaers_id INTEGER, 
symptom1 VARCHAR, 
symptomversion1 DECIMAL, 
symptom2 VARCHAR, 
symptomversion2 DECIMAL, 
symptom3 VARCHAR, 
sym

In [97]:
#closing connection

c.close()
    
conn.close()